In [152]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import plotly.io as pio


p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [153]:
sc.__version__

'1.8.1'

In [154]:
anndata.__version__

'0.8.0'

## 23/02/01 Checking adatas

#### Transcript metadata

In [155]:
var_cols = ['gene_ID', 'transcript_ID', 'annot_gene_id', 'annot_transcript_id',
            'annot_gene_name', 'annot_transcript_name', 'n_exons', 'gene_novelty',
            'transcript_novelty', 'ISM_subtype', 'length']
var_col = 'transcript_ID'

adata = sc.read('mouse_transcript.h5ad')
filt_ab = 'mouse_talon_abundance_filtered.tsv'
ab_df = pd.read_csv(filt_ab, sep='\t')

a_temp = adata.var[var_cols].sort_values(by=var_col).reset_index(drop=True)
ab_temp = ab_df[var_cols].sort_values(by=var_col).reset_index(drop=True)

cat_cols = a_temp.columns[a_temp.dtypes == 'category'].tolist()
for c in cat_cols:
    a_temp[c] = a_temp[c].astype('object')
    
pd.testing.assert_frame_equal(a_temp,
                              ab_temp)

#### Transcript expression

In [156]:
# get collapsed abundance table from adata
var_col = 'transcript_ID'
obs_col = 'dataset'
columns = adata.var[var_col].tolist()
rows = adata.obs[obs_col].tolist()
data = adata.X.toarray()

df = pd.DataFrame(index=rows, columns=columns, data=data)
df = df.transpose()
df.reset_index(inplace=True)
df['index'] = df['index'].astype('int')
df.rename({'index': var_col}, axis=1, inplace=True)
for c in df.columns:
    df[c] = df[c].astype('int64')

In [157]:
drop_cols = [c for c in var_cols if c != var_col]
drop_cols
temp = ab_df.drop(drop_cols, axis=1)
for c in temp.columns:
    temp[c] = temp[c].astype('int64')

In [158]:
pd.testing.assert_frame_equal(df,
                              temp)

In [159]:
# df.head()

In [160]:
# temp.head()

## Gene metadata

In [186]:
var_cols = ['gene_ID',  'annot_gene_id', 
            'annot_gene_name', 'gene_novelty']
var_col = 'gene_ID'

adata = sc.read('mouse_gene.h5ad')
filt_ab = 'mouse_talon_abundance.tsv'
ab_df = pd.read_csv(filt_ab, sep='\t')

a_temp = adata.var[var_cols].sort_values(by=var_col).reset_index(drop=True)
ab_temp = ab_df[var_cols].sort_values(by=var_col).reset_index(drop=True)
ab_temp.drop_duplicates(inplace=True)

# sort and deindex
a_temp = a_temp.sort_values(by=var_col).reset_index(drop=True)
ab_temp = ab_temp.sort_values(by=var_col).reset_index(drop=True)


cat_cols = a_temp.columns[a_temp.dtypes == 'category'].tolist()
for c in cat_cols:
    a_temp[c] = a_temp[c].astype('object')
    
pd.testing.assert_frame_equal(a_temp,
                              ab_temp) 

In [187]:
# get collapsed abundance table from adata
var_col = 'gene_ID'
obs_col = 'dataset'
columns = adata.var[var_col].tolist()
rows = adata.obs[obs_col].tolist()
data = adata.X.toarray()

df = pd.DataFrame(index=rows, columns=columns, data=data)
df = df.transpose()
df.reset_index(inplace=True)
df['index'] = df['index'].astype('int')
df.rename({'index': var_col}, axis=1, inplace=True)
for c in df.columns:
    df[c] = df[c].astype('int64')
    
# sort
df = df.sort_values(by='gene_ID')
df = df.reset_index(drop=True)

In [188]:
len(df.index)

252615

In [191]:
var_cols = ['gene_ID', 'transcript_ID', 'annot_gene_id', 'annot_transcript_id',
            'annot_gene_name', 'annot_transcript_name', 'n_exons', 'gene_novelty',
            'transcript_novelty', 'ISM_subtype', 'length']
drop_cols = [c for c in var_cols if c != var_col]
drop_cols
temp = ab_df.drop(drop_cols, axis=1)
for c in temp.columns:
    temp[c] = temp[c].astype('int64')

# sum ovr isos of the same gene
temp = temp.groupby('gene_ID').sum().reset_index()

# sort
temp = temp.sort_values(by='gene_ID')
temp = temp.reset_index(drop=True)

In [192]:
temp.head()
len(temp.index)

252615

In [193]:
# temp.set_index('gene_ID', inplace=True)
# temp.head()

In [194]:
# temp.loc[temp.any(axis=1)>0]

In [195]:
# temp.loc[temp.gene_ID.duplicated(keep =False)].sort_values(by='gene_ID').head()

In [196]:
df.head()

,gene_ID,gastroc_14d_f_2,gastroc_14d_f_1,heart_18-20mo_m_1,heart_18-20mo_m_2,heart_18-20mo_f_1,heart_18-20mo_f_2,adrenal_10d_f_1,adrenal_10d_f_2,adrenal_18-20mo_m_1,...,hippocampus_wt_m_2_1,hippocampus_wt_m_2_2,cortex_5x_f_1_1,cortex_5x_f_1_2,c2c12_myoblast_1_1,c2c12_myoblast_1_2,forelimb_e11_1_1,f1219_1_1,f1219_1_2,f1219_1_3
0,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,1,0,...,48,24,8,7,0,0,0,0,1,0
2,5,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,0,0,0,0,0,0,0,0,...,5,0,0,2,0,0,0,0,0,0


In [197]:
temp.head()

,gene_ID,gastroc_14d_f_2,gastroc_14d_f_1,heart_18-20mo_m_1,heart_18-20mo_m_2,heart_18-20mo_f_1,heart_18-20mo_f_2,adrenal_10d_f_1,adrenal_10d_f_2,adrenal_18-20mo_m_1,...,hippocampus_wt_m_2_1,hippocampus_wt_m_2_2,cortex_5x_f_1_1,cortex_5x_f_1_2,c2c12_myoblast_1_1,c2c12_myoblast_1_2,forelimb_e11_1_1,f1219_1_1,f1219_1_2,f1219_1_3
0,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,1,0,...,48,24,8,7,0,0,0,0,1,0
2,5,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,0,0,0,0,0,0,0,0,...,5,0,0,2,0,0,0,0,0,0


In [199]:
pd.testing.assert_frame_equal(df,
                              temp)

In [200]:
adata_gids = df.gene_ID.tolist()
ab_gids = temp.gene_ID.tolist()

In [201]:
print(len(adata_gids))
print(len(ab_gids))

252615
252615


In [202]:
len(list(set(adata_gids)&(set(ab_gids))))

252615

## devving getting all reference isos

In [ ]:
# get all known isoforms
# db = '../mouse/lr_bulk/talon/mouse.db'
db = 'mouse/lr_bulk/talon/mouse.db'
annot = 'gencode_vM21'

with sqlite3.connect(db) as conn:
    query = f"""SELECT DISTINCT t.gene_ID, t.transcript_ID
                    FROM transcripts as t
                    LEFT JOIN transcript_annotations as ta
                        ON ta.ID = t.transcript_ID
                    WHERE (ta.attribute = 'transcript_status'
                        AND ta.value = 'KNOWN'
                        AND ta.annot_name = '{annot}')
              """
    annot = pd.read_sql_query(query, conn)

In [ ]:
query = """SELECT DISTINCT gene_ID, transcript_ID FROM observed
               LEFT JOIN transcript_annotations AS ta
                   ON ta.ID = observed.transcript_ID
               WHERE (ta.attribute = 'transcript_status'
                      AND ta.value = 'KNOWN'
                      AND ta.annot_name = '%s')""" % (annot)
annot = pd.read_sql_query(query, conn)

In [ ]:
pass_list_file = 'mouse/lr_bulk/talon/mouse_vM21_novel_pass_list.csv'
